In [141]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd

if 'cachai' not in os.listdir('.'):
    os.chdir('../')
    
import cachai.utils.constants as C
from cachai.utils.models import BaseModel
from cachai.utils.metrics import evaluate, evaluate_group
from cachai.strategies.sanity_test import SanityTest

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [142]:
from sklearn.metrics import mean_absolute_error


class TTLSimulator:

    def __init__(self, iterations=1_000):
        self._iterations = iterations
        self._target_params = [
            (50, 5),
            (200, 10),
            (400, 30),
        ]
        # means = np.linspace(10, 500, 10).astype(int)
        # std = np.arange(1, len(means) + 1)**2
        # self._target_params = np.array([means, std]).T

    def update_target_params(self, progress):
        target_params = []
        for param in self._target_params:
            mean = float(
                round(param[0]*np.sin(progress*2*np.pi/2)/(param[0]/2) + param[0], 2)
            )
            std = param[1]
            target_params.append((mean, std))
        self._target_params = target_params

    def feedback(self, y_true, y_pred):
        observation_time = int(min(y_true, y_pred)[0])
        hits = max(0, observation_time-1)
        observation_type = None
        if y_pred[0] < y_true[0]:
            observation_type = C.MISS
        elif y_pred[0] > y_true[0]:
            observation_type = C.STALE
        else:
            observation_type = C.VALID_TTL
        return observation_time, observation_type, hits

    def generate(self):
        # get target
        target_param_index = np.random.randint(0, len(self._target_params))
        target_params = self._target_params[target_param_index]
        y = np.random.normal(target_params[0], target_params[1], 1)

        # generate features from target
        num_features = 1
        correlation = 0.8
        cov_matrix = np.eye(num_features) * (1 - correlation) + np.ones((num_features, num_features)) * correlation
        features = np.random.multivariate_normal(np.ones(num_features) * y, cov_matrix)
        X = features.reshape(1, -1)
        # X = np.full((1, num_features), fill_value=target)
        return X, y

In [143]:
class Experiment():

    DF_COLUMNS = [
        C.EXPERIMENT_NAME, C.MODEL_NAME, C.ITERATION, C.OBSERVATION_TYPE,
        C.OBSERVATION_TIME, C.Y_PRED, C.Y_TRUE, C.HITS, C.MAE
    ]

    def __init__(self, simulator, iterations=1_000):
        self._simulator = simulator
        self._iterations = iterations

    def run(self, experiments):
        df = []
        for experiment in experiments:
            experiment_name = experiment[C.EXPERIMENT_NAME]
            model = experiment[C.MODEL]
            debug = experiment[C.DEBUG]
            for i in range(self._iterations):
                X, y_true = self._simulator.generate()

                info = {C.Y_TRUE: y_true if debug else None}  # only for debugging
                y_pred = model.predict(X, info)
                observation_time, observation_type, hits = self._simulator.feedback(y_true, y_pred)
                model.observe(observation_time, observation_type, hits, y_pred, info)

                mae = mean_absolute_error(y_true, y_pred)
                df.append([
                    experiment_name, model.NAME, i, observation_type,
                    observation_time, y_pred[0], y_true[0], hits, mae
                ])
        return pd.DataFrame(df, columns=Experiment.DF_COLUMNS)

In [144]:
simulator = TTLSimulator()
experiment = Experiment(simulator=simulator)
df = experiment.run([
    {
        C.EXPERIMENT_NAME: 'Sanity test',
        C.MODEL: SanityTest(),
        C.DEBUG: True
    },
])
df.head(10)

,experiment_name,model_name,iteration,observation_type,observation_time,y_pred,y_true,hits,mae
0,Sanity test,SanityTest,0,valid_ttl,392,392.556064,392.556064,391,0.0
1,Sanity test,SanityTest,1,valid_ttl,199,199.288907,199.288907,198,0.0
2,Sanity test,SanityTest,2,valid_ttl,56,56.611305,56.611305,55,0.0
3,Sanity test,SanityTest,3,valid_ttl,395,395.527593,395.527593,394,0.0
4,Sanity test,SanityTest,4,valid_ttl,441,441.244396,441.244396,440,0.0
5,Sanity test,SanityTest,5,valid_ttl,423,423.933538,423.933538,422,0.0
6,Sanity test,SanityTest,6,valid_ttl,350,350.431683,350.431683,349,0.0
7,Sanity test,SanityTest,7,valid_ttl,204,204.725586,204.725586,203,0.0
8,Sanity test,SanityTest,8,valid_ttl,49,49.523306,49.523306,48,0.0
9,Sanity test,SanityTest,9,valid_ttl,223,223.385108,223.385108,222,0.0


In [145]:
evaluate_group(df)

,,,rmse,mae,mabe,msbe,hit_rate,miss_rate,precision,accuracy,recall,false_positive_rate,f1_score,hits_total,hits_mean,len
experiment_name,model_name,,,,,,,,,,,,,,,
Sanity test,SanityTest,0,0.0,0.0,0.0,0.0,0.0,1.0,0,1.0,0,0,0,212048,212.048,1000
